In [9]:
import random
import string
from hashlib import sha256
import time
import base64
import json

In [5]:
""" Python UGLIX client API
    Copyright (C) sysprog@UGLIX

    This program is free software: you can redistribute it and/or modify
    it under the terms of the UGL General Public License as published by
    the UGLIX Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; By running this program you implicitly agree
    that it comes without even the implied warranty of MERCHANTABILITY, 
    FITNESS FOR A PARTICULAR PURPOSE and you acknowledge that it may 
    potentially COMPLETELY DESTROY YOUR COMPUTER (even if it is unlikely), 
    INFECT IT WITH A VERY NASTY VIRUS or even RUN ARBITRARY CODE on it. 
    See the UGL (Uglix Public License) for more legal and technical details.
"""
import functools
import json
import urllib.request
import urllib.parse
import urllib.error


# Ceci est du code Python v3.4+ (une version >= 3.4 est requise pour une
# compatibilité optimale).

class ServerError(Exception):
    """
    Exception déclenchée en cas de problème côté serveur (URL incorrecte,
    accès interdit, requête mal formée, etc.)
    """
    def __init__(self, code=None, msg=None):
        self.code = code
        self.msg = msg

    def __str__(self):
        return "ERREUR {}, {}".format(self.code, self.msg)

class Connection:
    """
    Cette classe sert à ouvrir et à maintenir une connection avec le système
    UGLIX. Voir les exemples ci-dessous.

    Pour créer une instance de la classe, il faut spécifier une ``adresse de 
    base''. Les requêtes se font à partir de là, ce qui est bien pratique.
    L'adresse de base est typiquement l'adresse du système UGLIX.

    Cet objet Connection() s'utilise surtout via ses méthodes get(), post()...

    Il est conçu pour pouvoir être étendu facilement. En dériver une sous-classe
    capable de gérer des connexions chiffrées ne nécessite que 20 lignes de
    code supplémentaires.

    Exemple :
    >>> c = Connection()
    >>> c.get('/bin/echo')
    'usage: echo [arguments]'

    >>> print(c.get('/'))   # doctest: +ELLIPSIS
    HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL HAL
    ...
    """
    def __init__(self, base_url="http://isec.fil.cool/uglix"):
        self._base = base_url
        self._session = None   # au départ nous n'avons pas de cookie de session

    ############################################################################
    #                          MÉTHODES PUBLIQUES                              #
    ############################################################################
    def get(self, url):
        """
        Charge l'url demandée. Une requête HTTP GET est envoyée.

        >>> c = Connection()
        >>> c.get('/bin/echo')
        'usage: echo [arguments]'

        En cas d'erreur côté serveur, on récupère une exception.
        >>> c.get('/bin/foobar') # doctest: +ELLIPSIS
        Traceback (most recent call last):
        ...
        client.ServerError: ERREUR 404, ...
        """
        # prépare la requête
        request = urllib.request.Request(self._base + url, method='GET')
        return self._query(url, request)


    def post(self, url, **kwds):
        """
        Charge l'URL demandée. Une requête HTTP POST est envoyée. Il est 
        possible d'envoyer un nombre arbitraire d'arguments supplémentaires
        sous la forme de paires clef-valeur. Ces paires sont encodées sous la
        forme d'un dictionnaire JSON qui constitue le corps de la requête.

        Python permet de spécifier ces paires clef-valeurs comme des arguments
        nommés de la méthode post(). On peut envoyer des valeurs de n'importe
        quel type sérialisable en JSON.

        Par exemple, pour envoyer un paramètre nommé "string_example" de valeur
        "toto et un paramètre nommé "list_example" de valeur [True, 42, {'foo': 'bar'}],
        il faut invoquer :

        >>> c = Connection()
        >>> c.post('/bin/echo', string_example="toto", list_example=[True, 42, {'foo': 'bar'}])
        {'content_found': {'string_example': 'toto', 'list_example': [True, 42, {'foo': 'bar'}]}}

        L'idée est que la méthode post() convertit ceci en un dictionnaire JSON,
        qui ici ressemblerait à :

        {'string_example': 'toto', 'list_example': [True, 42, {'foo': 'bar'}]},

        puis l'envoie au serveur.
        """
        # prépare la requête
        request = urllib.request.Request(self._base + url, method='POST')
        data = None
        # kwds est un dictionnaire qui contient les arguments nommés. S'il
        # n'est pas vide, on l'encode en JSON et on l'ajoute au corps de la
        # requête.
        if kwds:     
            request.add_header('Content-type', 'application/json')
            data = json.dumps(kwds).encode()
        return self._query(url, request, data)


    def put(self, url, content):
        """
        Charge l'URL demandée avec une requête HTTP PUT. L'argument content
        forme le corps de la requête. Si content est de type str(), il est
        automatiquement encodé en UTF-8. cf /doc/strings pour plus de détails
        sur la question.
        """
        request = urllib.request.Request(self._base + url, method='PUT')
        if isinstance(content, str):
            content = content.encode()
        return self._query(url, request, data=content)

    ############################################################################
    #                     MÉTHODES PUBLIQUES AVANCÉES                          #
    ############################################################################


    def post_raw(self, url, data, content_type='application/octet-stream'):
        """
        Charge l'url demandée avec une requête HTTP POST. L'argument data
        forme le corps de la requête. Il doit s'agir d'un objet de type 
        bytes(). Cette méthode est d'un usage plus rare, et sert à envoyer des
        données qui n'ont pas vocation à être serialisées en JSON (comme des
        données binaires chiffrées, par exemple).

        Principalement utilisé pour étendre le client et lui ajouter des
        fonctionnalités.
        """
        request = urllib.request.Request(self._base + url, method='POST')
        request.add_header('Content-type', content_type)
        return self._query(url, request, data)

    def close_session(self):
        """
        Oublie la session actuelle. En principe, personne n'a besoin de ceci.
        """
        self._session = None


    ############################################################################
    #                          MÉTHODES INTERNES                               #
    ############################################################################

    def _pre_process(self, request):
        """
        Effectue un pré-traitement sur la requête pas encore lancée.
        Si on possède un cookie de session, on l'injecte dans les en-tête HTTP.
        """
        request.add_header('User-Agent', 'UGLIX official client v2.1 (c) sysprog')
        if self._session:
            request.add_header('Cookie', self._session)

    def _post_process(self, result, http_headers):
        """
        Effectue un post-traitement sur le résultat "brut" de la requête. En
        particulier, on décode les dictionnaires JSON, et on convertit le texte
        (encodé en UTF-8) en chaine de charactère Unicode. On peut étendre cette
        méthode pour gérer d'autres types de contenu si besoin.
        """
        if 'Content-Type' in http_headers:
            if http_headers['Content-Type'] == "application/json":
                return json.loads(result.decode())
            if http_headers['Content-Type'].startswith("text/plain"):
                return result.decode()
        # on ne sait pas ce que c'est : on laisse tel quel
        return result

    def _query(self, url, request, data=None):
        """
        Cette fonction à usage interne est appelée par get(), post(), put(),
        etc. Elle reçoit en argument une url et un objet Request() du module
        standard urllib.request.
        """
        self._pre_process(request)
        try:           
            # lance la requête. Si data n'est pas None, la requête aura un
            # corps non-vide, avec data dedans.
            with urllib.request.urlopen(request, data) as connexion:
                # récupère les en-têtes HTTP et le corps de la réponse, puis
                # ferme la connection
                headers = dict(connexion.info())
                result = connexion.read()
            
            # si on reçoit un identifiant de session, on le stocke
            if 'Set-Cookie' in headers:
                self._session = headers['Set-Cookie']

            # on effectue le post-processing, puis on renvoie les données.
            # c'est fini.
            return self._post_process(result, headers)

        except urllib.error.HTTPError as e:
            # On arrive ici si le serveur a renvoyé un code d'erreur HTTP
            # (genre 400, 403, 404, etc.). On récupère le corps de la réponse
            # car il y a peut-être des explications dedans. On a besoin des
            # en-tête pour le post-processing.
            headers = dict(e.headers)
            message = e.read()
            raise ServerError(e.code, self._post_process(message, headers)) from None


# vérifie l'authenticité de la signature du serveur
from zlib import decompress as Y
from base64 import b85decode as Z
public_key = 'V<bQ*EigZNBrR`4RU|-cFmOX>Fiv1-H&bLOcX>@%Mrm+jR$*2}ZA?f`cxgdjF?C' \
             'jWPB$YzDtbyiPftxZP&ipPDtlKpJ$NuyUR7&Ta4<YUdRTmEVKQ}TRB$9VC2=c6V' \
             'OLL1ZZSJ=D^O#7PE#u(ZC+SvUwm~pH#0DHVmwkkSVlKOO;~ABPJLoiS|mY9YIkv' \
             '7Jzh+Cb6zbbBT0EyMrnC|Y-&+@cQR#nMsjyaX?AN(awJAHNhmfWM_5!zEh=O+Jy' \
             'J$1M|yQoKXFcFMPFlDU~V@;Vof|NGb=neLpN_XZEs>(W+ZtuEmJ=<U|=aVC@pG0' \
             'G$cYdX(3^LZDDD1Pe(XWM@?^VYGP7vPD5y6A$?bRWlbSPQ$$K`T1ZwaJ}X#uVmN' \
             '9tW+pUsQZ#ZQM}9RzVl{esR#h}WR5L<7WnXMIQ(k3gST|rhLw0dtMl&!pMsY<cH' \
             '&a+sNLX_zdMGkEWOy_}cs^+(C^%6wa5PhRMkZ-KMqX%1K2<VmLR5GtD?Da$U?VA' \
             'bMKC!iHdkI>O=LeXUSm~RF-Rm)HDqB`bty}GXmxryZc<ZYKxtojKuIu6S8HZOXF' \
             'ygtV0SAmd3sDHJ49tBZ!vXLa8e_5UsG;<QbSp1R%cLTP%(RNG%{dfPh=z}J4j=9' \
             'J#IB@He+UJC1F!dGJ8H$SwJ~%STkvES#WM-C^J%IdSW&=B`97yVQ^JTb15o%U~y' \
             'JSXni4LV?tAQd|o6dR(NS)BUmL=Nj_CIB}g+uR%}CRPIhi|KYUMOHDY`|UQshtH' \
             'EeNKKru*AAy+jeHFPmjPkd2ia7bPuY$Q))XmD_Abwyw+Z9OY0Iaqa7CTcuob~Hb' \
             'CCN_6^RY74?cuz1XC`3n5es(@lFeX_yFf~?Dc0+G_PfKo9HcWj!P$XJAOFnUJJ8' \
             'msWdqjD2NP2t'
seed       = 'V<cc<Lr!ckd}m@*XfSXpFibpfKu<74MMg18D|9JAV{~ptbWCJbVLT>9Y;1HsH)j'
algorithm  = 'c%1E6%?^Sv41Sdz+gI32aAjT$$PPw~Mx$Xoy>3opbO`%VssahDw3N2rSDFm!VqI' \
             '1R%Q&;EoHGyrkY7*KN2lzNUK^+JY=rHhte1=VN-A&1kIK-XjgUpY4~fccguRs2T' \
             'QL~wdywrXFblvpq3h@Jr38m$;61jZ=P|rBT{S?MA&CA6iKI`GA>=4AQTy^tXm)=' \
             'KwLK;ioybCf>Bq#0CW1T`wNGjj++^U6f=v8Mb#y5Li0}W2aJ)kLk_t2=XDm6oCU' \
             'QP{9#KCf(<o$!`#-Zl*t^}HX>J=einE4oc1+#t8mFCIwOZdIsPm=EBjO3eu;k+'
signature = eval(Y(Z(algorithm)))
if not signature.startswith(b'S50|UU'):
    raise ValueError("ATTENTION : le serveur a été hacké !")

In [6]:
from subprocess import Popen, PIPE

# ce script suppose qu'il a affaire à OpenSSL v1.1.1
# vérifier avec "openssl version" en cas de doute.
# attention à MacOS, qui fournit à la place LibreSSL.

# en cas de problème, cette exception est déclenchée
class OpensslError(Exception):
    pass

# Il vaut mieux être conscient de la différence entre str() et bytes()
# cf /usr/doc/strings.txt


def encrypt(plaintext, passphrase, cipher='aes-128-cbc', base64=True):
    """invoke the OpenSSL library (though the openssl executable which must be
       present on your system) to encrypt content using a symmetric cipher.

       The passphrase is an str object (a unicode string)
       The plaintext is str() or bytes()
       The output is bytes()

       # encryption use
       >>> message = "texte avec caractères accentués"
       >>> c = encrypt(message, 'foobar')
    """
    # prépare les arguments à envoyer à openssl
    pass_arg = 'pass:{0}'.format(passphrase)
    if base64 == True:
        args = ['openssl', 'enc', '-' + cipher, '-base64', '-pass', pass_arg, '-pbkdf2']
    else:
        args = ['openssl', 'enc', '-' + cipher, '-pass', pass_arg, '-pbkdf2']
    
    # si le message clair est une chaine unicode, on est obligé de
    # l'encoder en bytes() pour pouvoir l'envoyer dans le pipeline vers 
    # openssl
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')
    
    # ouvre le pipeline vers openssl. Redirige stdin, stdout et stderr
    #    affiche la commande invoquée
    #    print('debug : {0}'.format(' '.join(args)))
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    
    # envoie plaintext sur le stdin de openssl, récupère stdout et stderr
    stdout, stderr = pipeline.communicate(plaintext)

    # si un message d'erreur est présent sur stderr, on arrête tout
    # attention, sur stderr on récupère des bytes(), donc on convertit
    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    # OK, openssl a envoyé le chiffré sur stdout, en base64.
    # On récupère des bytes, donc on en fait une chaine unicode
    if base64 ==True:
        return stdout.decode()
    else :
        return stdout


def decrypt(plaintext, passphrase, cipher='aes-128-cbc', base64=True):
    pass_arg = 'pass:{0}'.format(passphrase)
    if base64 == True:
        args = ['openssl', 'enc', '-' + cipher, '-base64', '-pass', pass_arg, '-pbkdf2',"-d"]
    else: 
        args = ['openssl', 'enc', '-' + cipher, '-pass', pass_arg, '-pbkdf2',"-d"]
        
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')

    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    
    stdout, stderr = pipeline.communicate(plaintext)

    error_message = stderr.decode()

    if error_message != '':
        raise OpensslError(error_message)
        
    if base64 ==True:
        return stdout.decode()
    else :
        return stdout
    #return stdout

In [7]:
def encrypt_public_key(plaintext, public_key):
    """
    chiffrer un texte avec une clef public.
    
    plaintext = string
    public_key = de la forme 'fichier.pem'
    """
    args = ['openssl', 'pkeyutl', '-encrypt', '-pubin','-inkey',public_key]
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')

    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate(plaintext)

    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)
        
    stdout = base64.b64encode(stdout)
    return stdout.decode()

def decrypt_public_key(plaintext, public_key):
    """
    dechiffrer avec la clef privé
    """
    args = ['openssl', 'pkeyutl', '-decrypt', '-inkey',public_key]
    if isinstance(plaintext, str):
        plaintext = base64.b64decode(plaintext)
        print("plaintext docoder ",plaintext)
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate(plaintext)

    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)
    
    return stdout.decode()


def gen_fichier_pem(fname, key, mode = 'w'):
    """
    key : string
    creer un fichier de nom fname contenant la clef public 
    """
    text_file = open(fname, mode)
    n = text_file.write(key)
    text_file.close()

def gen_keys(name_public, name_private):
    """
    generer public et private keys 
    creer les fichier .pem (if faut ecrire l'extension dans le nom)
    return un couple de string (public, private)
    """
    # genere une paire de clef 
    args = ['openssl', 'genpkey', '-algorithm','RSA', '-pkeyopt','rsa_keygen_bits:2048']
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate()
    stdout = stdout.decode()
    private = stdout   # clef privésoit moins de 2000 bits enviro
    
    gen_fichier_pem(name_private, private)
    
    #extraire la clef public
    text_file = open("tmp_gen_pkey.pem", "w")
    n = text_file.write(stdout)
    text_file.close()
    args = ['openssl', 'pkey', '-in', "tmp_gen_pkey.pem", '-pubout']
    pipe = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipe.communicate()
    public = stdout.decode()
    
    gen_fichier_pem(name_public, public)

    return public, private

In [ ]:
def prod_signature(secret_key, fichier_signa, msg):
    """
    secret_key nom fichier .pem
    fichier_signa nom du fichier dans laquel on va mettre la signature
    il fo les convertir en base 64
    """
    args = ['openssl', 'dgst', '-sha256', '-sign', secret_key]
    
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    
    if isinstance(msg, str):
        msg = msg.encode('utf-8')
    
    stdout, stderr = pipeline.communicate(msg)

    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)

    #signa = base64.b64encode(stdout)
    #gen_fichier_pem(fichier_signa, str(signa))
    
    gen_fichier_pem(fichier_signa, (stdout), "wb")
    return stdout


def prod_verify(public_key, fichier_signa, msg):
    """
    verifier la signature
    """
    args = ['openssl', 'dgst', '-sha256', '-verify', public_key, '-signature',fichier_signa]
    
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    
    if isinstance(msg, str):
        msg = msg.encode('utf-8')
    
    stdout, stderr = pipeline.communicate(msg)

    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)
        
    return stdout.decode()

#   ---- genere clefs et publie clef public --
#my_public_key,my_private_key =gen_keys('my_public_key.pem', 'my_private_key.pem')  
#print(c.post('/bin/key-management/upload-pk', public_key=my_public_key,confirm=True )) 
#   ---- fin ---------------------------------
#msg  = "testessssssssssssssssssssssssssssssssssssssssssssss"
#signa = prod_signature('my_private_key.pem', 'signature.bin', msg)

#prod_verify('my_public_key.pem', 'signature.bin', msg)

In [ ]:
def certi_affichage(certificat, mode = 'text'):
    args = ['openssl', 'x509', '-'+mode, '-noout']  
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate(certificat)
    error_message = stderr.decode()
    if error_message != '':
        raise OpensslError(error_message)
    return stdout.decode()

cert = b'-----BEGIN CERTIFICATE-----\nMIIB+DCCAZ2gAwIBAgIUNQ9yTqcMiuR+bbOZyxgAHcFim9owCgYIKoZIzj0EAwIw\nTjEkMCIGA1UECgwbSGlnaGVyLW9yZGVyIEFic3RyYWN0IExvZ2ljMQ4wDAYDVQQL\nDAVVR0xJWDEWMBQGA1UEAwwNVUdMSVggcm9vdCBDQTAeFw0yMDAyMjkxNDU4NDBa\nFw0yMTAyMjgxNDU4NDBaMEQxFzAVBgNVBAoMDkpvaG5zb24tTmd1eWVuMRcwFQYD\nVQQLDA5LZXkgTWFuYWdlbWVudDEQMA4GA1UEAwwHQmFuayBDQTBWMBAGByqGSM49\nAgEGBSuBBAAKA0IABGdCqynOYLvUls1R0gvLQQ3w7CgJ7i/lANd56UocdvMMecp9\nxQX9102PoNNKPWF0StjqV+f+fD4+Dh0V3xxIEr6jZjBkMB0GA1UdDgQWBBTV0xk0\ne1ixkw/gKkqsAwYxoUtgCjAfBgNVHSMEGDAWgBQ1DmjfeeCQ9E8NgdOGSFJL3XIY\niDASBgNVHRMBAf8ECDAGAQH/AgEAMA4GA1UdDwEB/wQEAwIBhjAKBggqhkjOPQQD\nAgNJADBGAiEApgJP28l7+WTYobg1VBfu3cl0tNHdBNkaL6rlE+CI2lwCIQD9Dpve\n8OX669ePbgmGOOWudgCOGgffDbD617BZ5TYFvQ==\n-----END CERTIFICATE-----\n'
#print(certi_affichage(cert))
#print(certi_affichage(cert,mode='subject'))
#print(certi_affichage(cert,mode='pubkey'))

def certi_verify(certificat):
    args = ['openssl', 'verify']
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate(certificat)
    error_message = stderr.decode()
    ##if error_message != '':
      ##  raise OpensslError(error_message)
    return stderr.decode()
#print(certi_verify(cert))


def certi_trusted(certificat, fic_certi_valide):
    args = ['openssl', 'verify', '-trusted', fic_certi_valide]  
    pipeline = Popen(args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = pipeline.communicate(certificat)
    error_message = stderr.decode()
    return stderr.decode()


In [ ]:
def passerelle_get(session_key, url, kerberos=False, num = 0):
    dico =json.dumps( {'method': 'GET', 'url': url})
    dico_encoded = encrypt(dico, session_key,base64=False)
#    dico_encoded = base64.b64decode(dico_encoded)
    
    if kerberos==True:
        retour = c.post_raw("/bin/uVM/"+str(num)+"/gateway", dico_encoded)
    else:
        retour = c.post_raw("/bin/gateway", dico_encoded)
    #retour = base64.b64encode(retour)+b'\n'  # ajout de \n car convention
    #retour = c.post_raw("/bin/gateway", dico_encoded)
    #retour = base64.b64encode(retour)+b'\n'  # ajout de \n car convention
    return decrypt(retour,session_key,base64=False).decode()
    
def passerelle_post(session_key, url, args):
    """
    args : {'foo': 42}
    """
    dico = json.dumps( {'method': 'POST', 'url': url, 'args': args})
    dico_encoded = encrypt(dico, session_key,base64=False)
    #dico_encoded = base64.b64decode(dico_encoded)
    
    retour = c.post_raw("/bin/gateway", dico_encoded)
    #retour = base64.b64encode(retour)+b'\n'  # ajout de \n car convention

    #retour = c.post_raw("/bin/gateway", dico_encoded)
    #retour = base64.b64encode(retour)+b'\n'  # ajout de \n car convention
    return decrypt(retour,session_key,base64=False).decode()

In [8]:
def dh():
    params = c.get('/bin/login/dh/parameters')
    p = params['p']
    g = params['g']
    certi = c.get('/bin/banks/CA')
    x = int('f00', base=16)
    A = pow(g,x,p)
    retour  = c.post("/bin/login/dh", username='theresagomez' ,A=A)
    B = retour['B']   # g**y mod p
    k = retour['k']
    signature = retour['signature']


    AB = pow(B,x,p)
    K = sha256((AB).to_bytes(256, byteorder='big'))
    session_key= K.hexdigest()
    print(session_key)

    T = str(A)+','+str(B)+','+str(k)+',UGLIX'
    sig = prod_signature('my_private_key.pem', 'signature.bin', T)
    sig = base64.b64encode(sig).decode()
    
    print(passerelle_post(session_key, '/bin/login/dh/confirmation',{'signature': sig}))
    return (session_key)

In [1]:
def kerberos(numMV):

    
    dic = (c.get("/bin/kerberos/authentication-service/theresagomez"))
    Client_TGS_session_key = dic['Client-TGS-session-key'] 
    session_key = decrypt(Client_TGS_session_key, '+JskVIss03')
    TGT = dic['TGT']
    
    d = {'username': 'theresagomez', 'timestamp': time.time()}
    e = json.dumps(d)  
    authenticator=encrypt(e, session_key)
    ticket = c.post("/bin/kerberos/ticket-granting-service",TGT=TGT,vm_name=numMV,authenticator=authenticator)
    
    
    Client_session_key=ticket['Client-Server-session-key']
    Client_session_key= decrypt(Client_session_key,session_key)
    Client_Server_ticket = ticket['Client-Server-ticket']
    
    d = {'username': 'theresagomez', 'timestamp': time.time()}
    e = json.dumps(d)
    authenticator=encrypt(e, Client_session_key)
    mv = c.post("/bin/uVM/"+str(numMV)+"/hello",ticket= Client_Server_ticket, authenticator=authenticator)
    return (Client_session_key)
    print(mv)

In [3]:
def erreur_decode(msg,key):
    print(decrypt(msg,key,base64=False))
#res = b'Salted__\xd6\xa6\xdf<\xf28G\x8b\xf4e\x82\xb7\x10\xd2Jw@\xd7\xe4\xab\xa2\xda/\xb2\xb4(\xa4\xba\xbe,Z)TQ\x89\xe8\xfa\x92\x1a$\xf0\x8bA\xce\x83\xb6\x01^;u\xf2\x1dE",s\x06\x9d\x90U\xd4\x7f\xf7\x06\xd5W\x02}A\x99\\\xf3'
#print(erreur_decode(res,Client_session_key))


In [ ]:
def stp (user = 'theresagomez', code='+JskVIss03'):
    nonce = c.post("/bin/login/stp", username =user )
    session_key = code+'-'+nonce
    print(passerelle_get(session_key, "/bin/login/stp/handshake"))
    return session_key